In [27]:
!python lm_eval \
        --model hf \
        --model_args pretrained=meta-llama/Llama-2-13b-chat-hf,parallelize=True \
        --tasks biolama_ctd \
        --device mps \
        --batch_size 1 \
        --output_path /Users/vincent/Documents/GP-NLP/lm-eval-log/llama-2-13b-chat-biolama-ctd \
        --include_path ./probing_tasks_vt \
        --log_samples\
        --limit 1 \
        --use_cache True \
        --gen_kwargs temperature=0


2024-03-10:19:11:42,399 INFO     [__main__.py:209] Verbosity set to INFO
2024-03-10:19:11:42,399 INFO     [__init__.py:358] lm_eval.tasks.initialize_tasks() is deprecated and no longer necessary. It will be removed in v0.4.2 release. TaskManager will instead be used.
2024-03-10:19:11:44,458 WARNING  [__main__.py:221]  --limit SHOULD ONLY BE USED FOR TESTING.REAL METRICS SHOULD NOT BE COMPUTED USING LIMIT.
2024-03-10:19:11:44,458 INFO     [__main__.py:226] Including path: ./probing_tasks_vt
2024-03-10:19:11:44,458 INFO     [__init__.py:349] To still use tasks loaded from args.include_path,see an example of the new TaskManager API in https://github.com/EleutherAI/lm-evaluation-harness/blob/main/docs/interface.md#external-library-usage
2024-03-10:19:11:44,458 INFO     [__main__.py:285] Selected Tasks: ['biolama_ctd']
2024-03-10:19:11:44,458 INFO     [__main__.py:286] Loading selected tasks...
2024-03-10:19:11:44,458 INFO     [evaluator.py:95] Setting random seed to 0
2024-03-10:19:11:44,4

In [26]:

!python lm_eval \
        --model hf \
        --model_args pretrained=EleutherAI/pythia-14m,parallelize=True \
        --tasks biolama_ctd \
        --device mps \
        --batch_size 2 \
        --output_path /Users/vincent/Documents/GP-NLP/lm-eval-log/pythia-14m-biolama-ctd \
        --include_path ./probing_tasks_vt \
        --log_samples\
        --limit 10 \
        --use_cache True \
        --gen_kwargs temperature=0


2024-03-10:19:08:36,160 INFO     [__main__.py:209] Verbosity set to INFO
2024-03-10:19:08:36,160 INFO     [__init__.py:358] lm_eval.tasks.initialize_tasks() is deprecated and no longer necessary. It will be removed in v0.4.2 release. TaskManager will instead be used.
2024-03-10:19:08:38,343 WARNING  [__main__.py:221]  --limit SHOULD ONLY BE USED FOR TESTING.REAL METRICS SHOULD NOT BE COMPUTED USING LIMIT.
2024-03-10:19:08:38,343 INFO     [__main__.py:226] Including path: ./probing_tasks_vt
2024-03-10:19:08:38,343 INFO     [__init__.py:349] To still use tasks loaded from args.include_path,see an example of the new TaskManager API in https://github.com/EleutherAI/lm-evaluation-harness/blob/main/docs/interface.md#external-library-usage
2024-03-10:19:08:38,344 INFO     [__main__.py:285] Selected Tasks: ['biolama_ctd']
2024-03-10:19:08:38,344 INFO     [__main__.py:286] Loading selected tasks...
2024-03-10:19:08:38,344 INFO     [evaluator.py:95] Setting random seed to 0
2024-03-10:19:08:38,3

 "Consider the following sentence: \"Bacterial Toxins prevents diseases such as <BLANK>.\"\n\n-> Which noun-phrase should <BLANK> be filled with? Give me 5 most probable candidates. Output your response in JSON format with keys \"top_1\", \"top_2\", \"top_3\", \"top_4\" and \"top_5\", where the value for key \"top_1\" is the most promising entity that would replace <BLANK>.

In [ ]:
from transformers import pipeline
from evaluate import load
from datasets import load_dataset


# Load the question answering pipeline with the desired model
qa_pipeline = pipeline("question-answering", model="meta-llama/Llama-2-13b-chat-hf")

# Load the evaluation dataset
dataset = load_dataset("CamWheeler135/Bio_Lama_TinyUMLS")

question = "Consider the following sentence: \"Bacterial Toxins prevents diseases such as <BLANK>.\"\n\n-> Which noun-phrase should <BLANK> be filled with? Give me 5 most probable candidates. Output your response in JSON format with keys \"top_1\", \"top_2\", \"top_3\", \"top_4\" and \"top_5\", where the value for key \"top_1\" is the most promising entity that would replace <BLANK>"

# Define the evaluation function
def evaluate_qa(examples):
    predictions = qa_pipeline(
        questions=question,
        top_k=5,  # Specify the value of k for top-k accuracy
    )

    # Extract the predicted nouns from the model's output
    predicted_nouns = [pred["answer"].lower() for pred in predictions]

    # Compare the predicted nouns with the ground truth answers
    accuracy = sum(1 for pred, gold in zip(predicted_nouns, examples["answers"]) if pred in gold) / len(examples["answers"])

    return {"accuracy": accuracy}

# Specify the value of k for top-k accuracy
k = 5

# Evaluate the model on the dataset
results = dataset.map(evaluate_qa, batched=True, batch_size=16)

# Print the overall top-k accuracy
print(f"Top-{k} Accuracy: {results['accuracy'].mean():.2f}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The model 'LlamaForCausalLM' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering', 'LEDForQuestionAnswering', 'LiltForQuestionAnswering', 'Longforme

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]